In [7]:
using Plots;
plotly();

if nprocs() < 3
    addprocs(3)
end

@everywhere function compute_pi(N::Int)
    # Aire d'un cercle: PI*r^2
    # Aire d'un carré: 4*r^2
    # Relation: PI / 4 = Ratio
    # et donc: Ratio*4 = PI
    # Ratio: Nombre dans le cercle / Nombre dans le carré
    in_circle = 0
    for i = 1:N
        
        X = rand(2) .* 2 .- 1
        d = sum(X.^2)
        
        if d < 1
            in_circle += 1
        end
    end

    return in_circle / N * 4.0    
end

function parallel_pi_computation(N::Int)    
    ncores = nprocs()-1

    sum_of_pis = @parallel (+) for i=1:ncores
        compute_pi(ceil(Int, N / ncores))
    end

    return sum_of_pis / ncores
end

plot(cos.(-pi:0.1:pi), sin.(-pi:0.1:pi), w=3)
x = rand(100) * 2 - 1
y = rand(100) * 2 - 1
scatter!(x, y, alpha=0.7)

In [28]:
n = 100000000

@time display(parallel_pi_computation(n))

@time display(compute_pi(n))

3.1416087771678245

3.14148788

  5.898291 seconds (1.28 k allocations: 81.906 KiB)
 15.837944 seconds (300.00 M allocations: 26.822 GiB, 25.02% gc time)


In [78]:
N = [i for i = 1:1000:500000]
P = map(parallel_pi_computation, N);

In [92]:
pies = [π for _ in N]
plot(N, pies, w=3, label="π", ylabel="Value", xlabel="n")
plot!(N, P, label="prediction")
ylims!((3.1, 3.2))